# The Finite Element Method applied to the 1D Poisson Equation

In the following, we will solve the **Poisson equation** subject to **Dirichlet boundary conditions**, i.e. we want to find a function $u: \Omega \subset \mathbb{R}^d \to \mathbb{R}$, which fulfills

\begin{equation}
    \begin{cases}
        -\Delta u(x) = f(x) & \text{if } x \in \operatorname{int} \Omega \\
        u(x) = g(x)         & \text{if } x \in \partial \Omega,
    \end{cases}
\end{equation}

where $$\Delta := \sum_{i = 1}^D \frac{\partial^2}{\partial x_i^2}$$ is the **Laplace operator**.
For simplicity, we set $\Omega = [l, r] \subset \mathbb{R}$, which means that the problem reduces to

$$
    \begin{cases}
        -u''(x) = f(x) & \text{for } x \in (l, r) \\
        u(x) = g(x) & \text{for } x \in \{l, r\}
    \end{cases}
$$

We will use the **finite element method** (**FEM**) to solve the problem numerically.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn
import scipy.linalg
import scipy.sparse
import scipy.sparse.linalg

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_poisson_equation_fem"
config.target = "imprs_2022"
config.debug_mode = True

In [ ]:
%matplotlib inline

In [ ]:
# Define the domain \Omega = [l, r]
domain = (-1.0, 1.0)
(l, r) = domain

## Closed-form Solution in 1D for $f(x) = c$

The 1D Poisson problem can be solved by simply integrating the PDE and enforcing the boundary conditions afterwards.
We have

\begin{align*}
    \int_l^x \int_l^{\nu_2} -u''(\nu_1) \text{d}\nu_1 \text{d}\nu_2
    & = - \int_l^x (u'(\nu_2) - u'(l)) \text{d}\nu_2 \\
    & = - (u(x) - u(l)) + u'(l) (x - l) \\
    & = -u(x) + u(l) + u'(l) (x - l)
\end{align*}

and

\begin{align*}
    \int_l^x \int_l^{\nu_2} f(\nu_1) \text{d}\nu_1 \text{d}\nu_2
    & = c \int_l^x \int_l^{\nu_2} \text{d}\nu_1 \text{d}\nu_2 \\
    & = c \int_l^x (\nu_2 - l) \text{d}\nu_2 \\
    & = \frac{c}{2} (x^2 - l^2) - cl(x - l) \\
    & = \frac{c}{2} (x - l)(x + l) - cl(x - l).
\end{align*}

This means that

\begin{align*}
    & -u(x) + u(l) + u'(l) (x - l) = \frac{c}{2} (x - l)(x + l) - c l(x - l) \\
    \Leftrightarrow \quad
    & u(x) = u(l) + \left( u'(l) + c l - \frac{c}{2} (x + l) \right) (x - l).
\end{align*}

Obviously $u(l) = g(l)$.
However, we still need to find $u'(l)$.
This can be done by enforcing the right boundary condition:

\begin{align*}
    & g(r) \stackrel{!}{=} u(r) = g(l) + \left( u'(l) + c l - \frac{c}{2} (r + l) \right) (r - l) \\
    \Leftrightarrow \quad
    & u'(l) (r - l) = g(r) - g(l) - c l (r - l) + \frac{c}{2} (r + l) (r - l) \\
    \Leftrightarrow \quad
    & u'(l) = \frac{g(r) - g(l)}{r - l} - c l + \frac{c}{2} (r + l),
\end{align*}

All in all, we arrive at

\begin{align*}
    u(x)
    & = g(l) + \frac{g(r) - g(l)}{r - l} (x - l) + \frac{c}{2} (r + l - (x + l)) (x - l) \\
    & = g(l) + \frac{g(r) - g(l)}{r - l} (x - l) - \frac{c}{2} (x - r)(x - l).
\end{align*}

In [ ]:
def poisson_1d_solution_constant_rhs(rhs, domain=(0.0, 1.0), boundary_values=(0.0, 0.0)):
    (l, r) = domain
    (g_l, g_r) = boundary_values
    
    aff_slope = (g_r - g_l) / (r - l)

    def u(x):
        return g_l + (aff_slope - (rhs / 2.0) * (x - r)) * (x - l)
    
    return u

In [ ]:
u_zero_boundary = poisson_1d_solution_constant_rhs(
    rhs=2.0,
    domain=domain,
    boundary_values=(0.0, 0.0),
)

In [ ]:
xs_plot = np.linspace(l, r, 100)

plt.plot(xs_plot, u_zero_boundary(xs_plot))
plt.show()

## Finite Element Solution for $g(x) = 0$

For the sake of simplicity, we will now assume that $g(x) = 0$ for $x \in \partial \Omega$.

### Step 1: Weak Formulation

To apply the finite element method, we must first convert the problem to its so-called **weak formulation**.

Let $V$ be a vector space of sufficiently smooth functions $\Omega \to \mathbb{R}$ with $v(x) = 0$ for all $v \in V$.
Note that a function $u \in V$ solves the problem above if and only if $\langle -\Delta u, v \rangle = \langle f, v \rangle$, or equivalently
$$-\int_\Omega \Delta u(x) v(x) \text{d}x = \int_\Omega f(x) v(x) \text{d}x$$
for every $v \in V$.
We can introduce additional symmetry to this formulation by applying Green's first identity to the left-hand side of the equation
$$-\int_\Omega \Delta u(x) v(x) \text{d}x = \int_\Omega \nabla u(x) \cdot \nabla v(x) \text{d}x - \int_{\partial \Omega} \underbrace{v(x)}_{= 0} (\nabla u(x) \cdot \mathbf{n}) \text{d}S = \int_\Omega \nabla u(x) \cdot \nabla v(x) \text{d}x.$$
This means that our original problem can be solved by finding a function $u \in V$ such that
$$\int_\Omega \nabla u(x) \nabla v(x) \text{d}x = \int_\Omega f(x) v(x) \text{d}x$$
for all $v \in V$.
This is commonly referred to as the weak formulation of the Poisson problem.

Note that we did not yet define the function space $V$ precisely.
In order to be able to write down the equations above, we require that the first (weak) derivatives of $u$ and $v$ are square integrable.
This means that an appropriate choice of $V$ is $V = H_0^1(\Omega)$, the Sobolev space of functions with square-integrable first (weak) derivatives which attain the value 0 on the boundary of $\Omega$.

In the 1D case, the integral equation simplifies to
$$\int_l^r u'(x) v'(x) \text{d}x = \int_l^r f(x) v(x) \text{d}x.$$

### Step 2: Discretization

Next, we convert the continuous problem above into a discrete problem by replacing $V$ with a finite-dimensional subspace $\hat{V} \subset V$.

#### The FEM Basis

In the one-dimensional finite element method, typically constucts $\hat{V}$ as follows:

The input domain $\Omega = [l, r]$ is discretized into a grid $(x_i)_{i = 1}^{n + 2}$ with $l = x_0 \le x_1 \le \dots \le x_{n + 1} = r$.

In [ ]:
n = 9
grid = np.linspace(l, r, n + 2)

The grid can then be used to define a set of $n$ basis functions for $\hat{V}$:
$$
    \phi_i(x) :=
    \begin{cases}
        \frac{x - x_{i - 1}}{x_i - x_{i - 1}} & \text{if } x \in [x_{i - 1}, x_i] \\
        \frac{x_{i + 1} - x}{x_{i + 1} - x_i} & \text{if } x \in [x_i, x_{i + 1}] \\
        0 & \text{otherwise}
    \end{cases},
$$
where $i = 1, \dots, n$.
Note that this basis is constructed such that $\hat{V}$ only contains functions which directly fulfill the boundary conditions.

In [ ]:
def plot_1d_fem_basis_zero_boundary(ax, grid, coords=None, **plot_kwargs):
    """Assumes an ordered grid"""
    if coords is None:
        coords = np.ones_like(grid[1:-1])
    
    xs = np.vstack((grid[:-2], grid[1:-1], grid[2:]))

    ys = np.empty_like(xs)
    ys[0, :] = 0.0
    ys[1, :] = coords
    ys[2, :] = 0.0

    ax.plot(xs, ys, **plot_kwargs)

In [ ]:
plot_1d_fem_basis_zero_boundary(plt.gca(), grid)

### Span of the FEM Basis

The function space spanned by this set of basis functions is given by
$$\hat{V} = \{ f_w := \sum_{i = 1}^n w_i \phi_i \mid w \in \mathbb{R}^n \}.$$

To see how $f_w$ behaves, let $x \in [x_k, x_{k + 1}]$ for $k = 1, \dots, n - 1$.
Then

\begin{align*}
    f_w(x)
    & = \sum_{i = 1}^n w_i \phi_i(x) \\
    & = \sum_{i = 1}^n w_i
    \begin{cases}
        \frac{x - x_{i - 1}}{x_i - x_{i - 1}} & \text{if } x \in [x_{i - 1}, x_i] \\
        \frac{x_{i + 1} - x}{x_{i + 1} - x_i} & \text{if } x \in [x_i, x_{i + 1}] \\
        0 & \text{otherwise}
    \end{cases}
    \\
    & = w_k \frac{x_{k + 1} - x}{x_{k + 1} - x_k} + w_{k + 1} \frac{x - x_{(k + 1) - 1}}{x_{k + 1} - x_{(k + 1) - 1}} \\
    & = \frac{x_{k + 1} - x_k + x_k - x}{x_{k + 1} - x_k} w_k + \frac{x - x_k}{x_{k + 1} - x_k} w_{k + 1} \\
    & = \left( 1 - \frac{x - x_k}{x_{k + 1} - x_k} \right) w_k + \frac{x - x_k}{x_{k + 1} - x_k} w_{k + 1}.
\end{align*}

We can see that $f_w$ attains value $w_i$ at grid point $i$ and it interpolates linearly in between grid points.

In [ ]:
def fem_zero_boundary_coords_to_fn(grid, coords):
    ys_grid = np.empty_like(coords, shape=(grid.shape[0],))
    ys_grid[0] = 0.0
    ys_grid[1:-1] = coords
    ys_grid[-1] = 0.0

    return lambda x: np.interp(x, grid, ys_grid)

In [ ]:
w = u_zero_boundary(grid[1:-1])

xs_plot = np.linspace(l, r, 100)

plt.plot(xs_plot, fem_zero_boundary_coords_to_fn(grid, w)(xs_plot), label="$f_w$")
plot_1d_fem_basis_zero_boundary(plt.gca(), grid, coords=w, alpha=0.2)
plt.legend()
plt.show()

### Weak Formulation of the Poisson Problem in the FEM basis

We can now express $u$ in the weak formulation by its basis expansion
$$u(x) = \sum_{j = 1}^n \hat{u}_j \phi_j(x),$$
i.e.
$$u'(x) = \sum_{j = 1}^n \hat{u}_j \phi_j'(x).$$
This means that we now need to find the coefficients $u_i$ such that
$$\sum_{j = 1}^n \hat{u}_j \int_l^r \phi_j'(x) v'(x) \text{d}x = \int_l^r f(x) v(x) \text{d}x$$
for all $v \in \hat{V}$.
Since the equation is linear in $v$, this is equivalent to solving the system of equations
$$\sum_{j = 1}^n \hat{u}_j \int_l^r \phi_j'(x) \phi_i'(x) \text{d}x = \int_l^r f(x) \phi_i(x) \text{d}x,$$
where $j = 1, \dots, n$ for the coeffients $\hat{u}_i$.

If we define a matrix $A \in \mathbb{R}^{n \times n}$ with
$$A_{ij} := \int_l^r \phi_i'(x) \phi_j'(x) \text{d}x,$$
and a vector $b \in \mathbb{R}^n$, where
$$b_i := \int_l^r \phi_i(x) f(x) \text{d}x,$$
we can equivalently write the weak formulation on $\hat{V}$ as
$$(A \hat{u})_i = \sum_{j = 1}^n \left( \int_l^r \phi_i'(x) \phi_j'(x) \text{d}x \right) \hat{u}_j = \int_l^r f(x) \phi_i(x) \text{d}x = b_i.$$
Hence, we have now reduced our original continuous Poisson problem to a linear system $A \hat{u} = b$.

### Computing Closed-form Expressions for $A$ and $b$

We will now compute closed-form expressions for the entries of $A$.
First of all, note that $A$ is symmetric.
Moreover, we have
$$
    \phi_i'(x) =
    \begin{cases}
        \frac{1}{x_i - x_{i - 1}}, & \text{if } x \in [x_i - x_{i - 1}] \\
        -\frac{1}{x_{i + 1} - x_i}, & \text{if } x \in [x_{i + 1} - x_i] \\
        0, & \text{otherwise}
    \end{cases}
$$
(in the weak sense).
This implies that $A_{ij} = 0$ for $j \notin \{ i - 1, i, i + 1 \}$, since the support of $\phi_i'$ only overlaps with the support of $\phi_j'$ for $j \in \{ i - 1, i, i + 1 \}$.
Hence, $A$ is tridiagonal.

Consequently, we only need to compute $A_{ii}$ for $i = 1, \dotsc, n$, and $A_{i,i+1}$ for $i = 1, \dots, n - 1$.

\begin{align*}
    A_{ii}
    & := \int_l^r (\phi_i'(x))^2 \text{d}x \\
    & = \int_{x_{i - 1}}^{x_i} \left( \frac{1}{x_i - x_{i - 1}} \right)^2 \text{d}x + \int_{x_i}^{x_{i + 1}} \left( -\frac{1}{x_{i + 1} - x_i} \right)^2 \text{d}x \\
    & = \frac{1}{(x_i - x_{i - 1})^2} \int_{x_{i - 1}}^{x_i} \text{d}x + \frac{1}{(x_{i + 1} - x_i)^2} \int_{x_i}^{x_{i + 1}} \text{d}x \\
    & = \frac{x_i - x_{i - 1}}{(x_i - x_{i - 1})^2} + \frac{x_{i + 1} - x_i}{(x_{i + 1} - x_i)^2} \\
    & = \frac{1}{x_i - x_{i - 1}} + \frac{1}{x_{i + 1} - x_i}
\end{align*}

\begin{align*}
    A_{i,i + 1}
    & := \int_l^r \phi_i'(x) \phi_{i + 1}'(x) \text{d}x \\
    & = \int_{x_{i - 1}}^{x_i} \frac{1}{x_i - x_{i - 1}} \cdot 0 \text{d}x \\
    \qquad & + \int_{x_i}^{x_{i + 1}} \left( -\frac{1}{x_{i + 1} - x_i} \right) \left( \frac{1}{x_{i + 1} - x_i} \right) \text{d}x \\
    \qquad & + \int_{x_{i + 1}}^{x_{i + 2}} 0 \cdot \left( -\frac{1}{x_{i + 1} - x_i} \right) \text{d}x \\
    & = -\frac{1}{(x_{i + 1} - x_i)^2} \int_{x_i}^{x_{i + 1}} \text{d}x \\
    & = -\frac{x_{i + 1} - x_i}{(x_{i + 1} - x_i)^2} \\
    & = -\frac{1}{x_{i + 1} - x_i}
\end{align*}

In [ ]:
def poisson_1d_zero_boundary_operator_fem(grid: np.ndarray) -> pn.linops.Matrix:
    # Diagonal
    diag = 1 / (grid[1:-1] - grid[:-2])
    diag += 1 / (grid[2:] - grid[1:-1])

    # Off-Diagonals
    offdiag = -1.0 / (grid[2:-1] - grid[1:-2])
    
    return pn.linops.Matrix(
        scipy.sparse.diags(
            (offdiag, diag, offdiag),
            offsets=(-1, 0, 1),
            format="csr",
            dtype=np.double,
        )
    )

In [ ]:
A = poisson_1d_zero_boundary_operator_fem(grid)

# Plot a heatmap of the matrix
vmax = np.max(np.abs(A.todense()))

plt.imshow(A.todense(), cmap="bwr", vmin=-vmax, vmax=vmax)
plt.colorbar()
plt.show()

In order to find a closed-form expression for the right-hand side, we either need to fix a closed-form representation of $f$ or revert to quadrature algorithms.

For the sake of simplicity, we will assume $f(x) = c \in \mathbb{R}$ in the following.
In this case, the entries of $b$ are given by

\begin{align*}
    b_i
    & = \int_l^r f(x) \phi_i(x) \text{d}x \\
    & = c \int_l^r \phi_i(x) \text{d}x \\
    & = c \left( \int_{x_{i - 1}}^{x_i} \frac{x - x_{i - 1}}{x_i - x_{i - 1}} \text{d}x + \int_{x_i}^{x_{i + 1}} \frac{x_{i + 1} - x}{x_{i + 1} - x_i} \text{d}x \right) \\
    & = c \left( \frac{1}{x_i - x_{i - 1}} \int_{x_{i - 1}}^{x_i} x - x_{i - 1} \text{d}x + \frac{1}{x_{i + 1} - x_i} \int_{x_i}^{x_{i + 1}} x_{i + 1} - x \text{d}x \right) \\
    & = c \left( \frac{1}{x_i - x_{i - 1}} \int_0^{x_i - x_{i - 1}} x \text{d}x + \frac{1}{x_{i + 1} - x_i} \int_{x_{i + 1} - x_i}^0 x \cdot (-1) \text{d}x \right) \\
    & = c \left( \frac{1}{x_i - x_{i - 1}} \left( \frac{1}{2} x^2 \bigg \rvert_0^{x_i - x_{i - 1}} \right) - \frac{1}{x_{i + 1} - x_i} \left( \frac{1}{2} x^2 \bigg \rvert_{x_{i + 1} - x_i}^0 \right) \right) \\
    & = \frac{c}{2} \left( \frac{(x_i - x_{i - 1})^2}{x_i - x_{i - 1}} + \frac{(x_{i + 1} - x_i)^2}{x_{i + 1} - x_i} \right) \\
    & = \frac{c}{2} (x_i - x_{i - 1} + (x_{i + 1} - x_i)) \\
    & = \frac{c}{2} (x_{i + 1} - x_{i - 1}).
\end{align*}

In [ ]:
c = 2.0

In [ ]:
def poisson_1d_rhs_fem(rhs: float, grid: np.ndarray) -> np.ndarray:
    if isinstance(rhs, float):
        return (rhs / 2) * (grid[2:] - grid[:-2])
    else:
        raise TypeError

In [ ]:
b = poisson_1d_rhs_fem(c, grid)
b

### Step 3: Solving the Linear System

In the final step of the finite element method, we need to solve the linear system $A \hat{u} = b$, where $A$ and $b$ are defined as above.

In [ ]:
A = poisson_1d_zero_boundary_operator_fem(grid)
b = poisson_1d_rhs_fem(c, grid)

In [ ]:
fig, axes = plt.subplots(
    ncols=3,
    figsize=(5, 3.5),
    gridspec_kw={
        "width_ratios": [4, 0.75, .2]
    },
)

vmax = np.max(np.abs(np.hstack([A.todense(), b[:, None]])))

img = axes[0].imshow(A.todense(), cmap="bwr", vmin=-vmax, vmax=vmax)
axes[1].imshow(b[:, None], cmap="bwr", vmin=-vmax, vmax=vmax)
fig.colorbar(img, cax=axes[2])

for ax in axes[:-1]:
    ax.set_xticks([])
    ax.set_yticks([])

fig.tight_layout()
plt.show()

The system matrix $A$ is sparse which means that we can benefit from a sparse solver like the method of **conjugate gradients**.

In [ ]:
u_zero_boundary_fem_coords, _ = scipy.sparse.linalg.cg(A.A, b)

In [ ]:
u_zero_boundary_fem = fem_zero_boundary_coords_to_fn(grid, u_zero_boundary_fem_coords)

In [ ]:
xs_plot = np.linspace(l, r, 100)

plt.plot(xs_plot, u_zero_boundary(xs_plot), label="Exact Solution")
plt.plot(xs_plot, u_zero_boundary_fem(xs_plot), label="FEM Solution")
plot_1d_fem_basis_zero_boundary(plt.gca(), grid, coords=u_zero_boundary_fem_coords, alpha=0.2)
plt.legend()
plt.show()

### Implementation in `linpde_gp`

In [ ]:
import ipywidgets

In [ ]:
%matplotlib widget

fig, ax = plt.subplots(num="Solution to the 1D Poisson Problem with g(x) = 0")

def interact(domain: tuple, rhs: float, n: int):
    # Problem definition
    bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(
        domain=domain,
        rhs=linpde_gp.functions.Constant(input_shape=(), value=rhs),
        boundary_values=(0.0, 0.0),
    )
    
    # FEM Discretization
    basis = linpde_gp.galerkin.bases.ZeroBoundaryFiniteElementBasis(
        domain=bvp.domain,
        num_elements=n,
    )
    linsys = linpde_gp.galerkin.project(bvp, basis)
    
    # Solve with CG
    solver = linpde_gp.linalg.solvers.ConjugateGradients()
    u_fem_coords = solver.solve(linsys)
    
    # FEM Solution
    u_fem = basis.coords2fn(u_fem_coords)
    
    # Plotting
    plot_grid = np.linspace(*domain, 200)
    
    ax.cla()
    ax.plot(plot_grid, bvp.solution(plot_grid), label="Exact Solution")
    ax.plot(plot_grid, u_fem(plot_grid).support, label="FEM Solution")
    ax.legend()

    fig.canvas.draw()
    
ipywidgets.interactive(
    interact,
    domain=ipywidgets.FloatRangeSlider(
        value=(-1.0, 1.0),
        min=-3.0,
        max=3.0,
        description="Domain",
    ),
    rhs=ipywidgets.FloatSlider(
        value=2.0,
        min=-3.0,
        max=3.0,
        description="f(x)",
    ),
    u_l=ipywidgets.FloatSlider(
        value=0.0,
        min=-2.0,
        max=2.0,
        description="g(l)",
    ),
    u_r=ipywidgets.FloatSlider(
        value=0.0,
        min=-2.0,
        max=2.0,
        description="g(r)",
    ),
    n=ipywidgets.IntSlider(
        value=1,
        min=1,
        max=20,
        continuous_update=True,
    ),
)

In [ ]:
%matplotlib inline

## Finite Element Solution for Non-zero Boundary Conditions

In [ ]:
boundary_values = (0.0, 1.0)

In [ ]:
n = 3
grid = np.linspace(l, r, n + 2)

In [ ]:
def discrete_poisson_problem_with_boundary_conditions(grid):
    (N,) = grid.shape
    
    diag = np.empty_like(grid)
    offdiag = np.empty_like(grid, shape=(N - 1,))

    # Laplace Operator on the interior
    diag[1:-1] = (
        1 / (grid[1:-1] - grid[:-2])
        + 1 / (grid[2:] - grid[1:-1])
    )
    offdiag[1:-1] = -1.0 / (grid[2:-1] - grid[1:-2])
    
    # Left boundary condition
    diag[0] = 1.0
    offdiag[0] = 0.0
    
    # Right boundary condition
    diag[-1] = 1.0
    offdiag[-1] = 0.0
    
    return pn.linops.Matrix(
        scipy.sparse.diags(
            (offdiag, diag, offdiag),
            offsets=(-1, 0, 1),
            format="csr",
            dtype=grid.dtype,
        )
    )

In [ ]:
plt.imshow(discrete_poisson_problem_with_boundary_conditions(grid).todense())
plt.colorbar()

In [ ]:
def discrete_poisson_problem_with_boundary_conditions_rhs(grid, alpha, boundary_values):
    rhs = np.empty_like(grid)
    
    l, r = boundary_values
    
    rhs[1:-1] = (alpha / 2) * (grid[2:] - grid[:-2])
    
    # Left Boundary Condition
    rhs[0] = l
    rhs[1] += l / (grid[2] - grid[1])
    
    # Right Boundary Condition
    rhs[-1] = r
    rhs[-2] += r / (grid[-2] - grid[-3])

    return rhs

In [ ]:
discrete_poisson_problem_with_boundary_conditions_rhs(grid, 1.0, boundary_values=boundary_values)

In [ ]:
def discrete_poisson_problem_with_boundary_conditions_sol(grid, boundary_values):
    A = discrete_poisson_problem_with_boundary_conditions(grid)
    b = discrete_poisson_problem_with_boundary_conditions_rhs(grid, 2.0, boundary_values=boundary_values)

    (u, _) = scipy.sparse.linalg.cg(A.A, b)
    
    return u

In [ ]:
sol = discrete_poisson_problem_with_boundary_conditions_sol(grid, boundary_values)

In [ ]:
sol

In [ ]:
with plt.rc_context(config.tueplots_bundle()):
    plt.plot(grid, sol, label="FEM Solution")
    plt.plot(
        xs_plot,
        poisson_1d_solution_constant_rhs(2.0, domain, boundary_values)(xs_plot),
        label="Exact Solution",
    )
    plt.legend()
    
experiment_utils.savefig("poisson_1d_fem")

### Comparison with clean implementation

In [ ]:
import ipywidgets

%matplotlib widget

fig, ax = plt.subplots(num="Solution to the 1D Poisson Problem")

def interact(domain: tuple, rhs: float, u_l: float, u_r: float, n: int):
    bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(
        domain=domain,
        rhs=linpde_gp.functions.Constant(input_shape=(), value=rhs),
        boundary_values=(u_l, u_r),
    )
    
    # FEM Discretization
    basis = linpde_gp.galerkin.bases.FiniteElementBasis(
        domain=bvp.domain,
        boundary_conditions=bvp.boundary_conditions,
        num_elements=n,
    )
    linsys = linpde_gp.galerkin.project(bvp, basis)
    
    # Solve with CG
    solver = linpde_gp.linalg.solvers.ConjugateGradients()
    u_fem_coords = solver.solve(linsys)

    # FEM Solution
    u_fem = basis.coords2fn(u_fem_coords)
    
    # Plotting
    plot_grid = np.linspace(*domain, 200)
    
    ax.cla()
    ax.plot(plot_grid, bvp.solution(plot_grid), label="Exact Solution")
    ax.plot(plot_grid, u_fem(plot_grid).support, label="FEM Solution")
    ax.legend()

    fig.canvas.draw()
    
ipywidgets.interactive(
    interact,
    domain=ipywidgets.FloatRangeSlider(
        value=(-1.0, 1.0),
        min=-3.0,
        max=3.0,
        description="Domain",
    ),
    rhs=ipywidgets.FloatSlider(
        value=2.0,
        min=-3.0,
        max=3.0,
        description="f(x)",
    ),
    u_l=ipywidgets.FloatSlider(
        value=0.0,
        min=-2.0,
        max=2.0,
        description="g(l)",
    ),
    u_r=ipywidgets.FloatSlider(
        value=0.0,
        min=-2.0,
        max=2.0,
        description="g(r)",
    ),
    n=ipywidgets.IntSlider(
        value=1,
        min=1,
        max=20,
        continuous_update=True,
    ),
)